<a href="https://colab.research.google.com/github/ayachaieb/fruit-detection-and-counting/blob/main/yolomodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi


Tue Jun 18 04:24:37 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
# Pip install method (recommended)

!pip install ultralytics==8.0.20

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 30.2/78.2 GB disk)


In [ ]:
from ultralytics import YOLO

from IPython.display import display, Image

In [ ]:

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/my cv/dataset_fruit_detection

/content/drive/MyDrive/my cv/dataset_fruit_detection


In [ ]:
import os
import shutil
import pandas as pd
import random

# Set up the source and destination paths
source_images_dir = "/content/drive/MyDrive/acfr-fruit-dataset/apples/images"
source_annotations_dir = "/content/drive/MyDrive/acfr-fruit-dataset/apples/annotations"
destination_dir = "apples_yolo"  # Create this directory if it doesn't exist

# Create destination directory
os.makedirs(destination_dir, exist_ok=True)

# Function to convert annotations
def convert_annotations(annotation_file):
  df = pd.read_csv(annotation_file)
  df['x_min'] = df['c-x'] - df['radius']
  df['y_min'] = df['c-y'] - df['radius']
  df['x_max'] = df['c-x'] + df['radius']
  df['y_max'] = df['c-y'] + df['radius']
  df = df[['x_min', 'y_min', 'x_max', 'y_max', 'label']]
  return df

# Create train/valid/test split
def create_split(images, train_ratio=0.8, valid_ratio=0.1, test_ratio=0.1):
  """Creates a train/valid/test split of the image data.

  Args:
    images: List of image filenames.
    train_ratio: Ratio of images for training.
    valid_ratio: Ratio of images for validation.
    test_ratio: Ratio of images for testing.

  Returns:
    A tuple containing three lists: train_images, valid_images, test_images.
  """

  random.shuffle(images)
  total_images = len(images)
  train_size = int(total_images * train_ratio)
  valid_size = int(total_images * valid_ratio)
  test_size = int(total_images * test_ratio)

  train_images = images[:train_size]
  valid_images = images[train_size:train_size + valid_size]
  test_images = images[train_size + valid_size:]

  return train_images, valid_images, test_images

# Get image filenames
image_filenames = [f for f in os.listdir(source_images_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]

# Create train/valid/test split
train_images, valid_images, test_images = create_split(image_filenames)

# Write split files
with open(os.path.join("apples_yolo", "train.txt"), "w") as f:
  for image in train_images:
    f.write(f"{image}\n")

with open(os.path.join("apples_yolo", "valid.txt"), "w") as f:
  for image in valid_images:
    f.write(f"{image}\n")

with open(os.path.join("apples_yolo", "test.txt"), "w") as f:
  for image in test_images:
    f.write(f"{image}\n")

# Copy images and convert annotations
for image_filename in image_filenames:
  image_path = os.path.join(source_images_dir, image_filename)
  annotation_path = os.path.join(source_annotations_dir, image_filename[:-4] + ".csv")

  # Copy image
  new_image_path = os.path.join(destination_dir, image_filename)
  shutil.copyfile(image_path, new_image_path)

  # Convert annotations
  yolo_annotations = convert_annotations(annotation_path)
  yolo_annotation_path = os.path.join(destination_dir, image_filename[:-4] + ".txt")
  yolo_annotations.to_csv(yolo_annotation_path, index=False, header=False, sep=" ")

In [ ]:

%cd /content/drive/MyDrive/my cv/dataset_fruit_detection

!yolo task=detect mode=train model=yolov8s.pt data= data.yaml epochs=20 imgsz=224 plots=True

/content/drive/MyDrive/my cv/dataset_fruit_detection
Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8s.yaml, data=data.yaml, epochs=20, patience=50, batch=16, imgsz=224, save=True, cache=False, device=, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, overlap_mask=True, mask_ratio=4, dropout=False, val=True, save_json=False, save_hybrid=False, conf=0.001, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=ultralytics/assets/, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_conf=False, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False, int8=Fals

In [ ]:
!yolo task=detect mode=val model=runs/detect/train2/weights/best.pt data=data.yaml

2024-06-18 05:22:42.725912: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-18 05:22:42.725963: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-18 05:22:42.727247: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-18 05:22:42.735092: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-18 05:22:43.905201: W tensorflow/compiler/tf2

In [ ]:
!yolo task=detect mode=predict model=runs/detect/train2/weights/best.pt source='/content/drive/MyDrive/my cv/dataset_fruit_detection/test/images'

2024-06-18 05:25:14.382033: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-18 05:25:14.382094: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-18 05:25:14.383538: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-18 05:25:14.390992: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-18 05:25:15.644437: W tensorflow/compiler/tf2

In [ ]:
!yolo task=detect mode=predict model=runs/detect/train2/weights/best.pt source='/content/apple.jpg'

2024-06-18 05:35:11.597361: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-18 05:35:11.597430: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-18 05:35:11.599369: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-18 05:35:11.610452: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-18 05:35:13.369810: W tensorflow/compiler/tf2